In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


No GPU was detected. LSTMs and CNNs can be very slow without a GPU.
Go to Runtime > Change runtime and select a GPU hardware accelerator.


In [2]:
tf.random.set_seed(42)

# RNN Implementation

Starter code taken from readings provided to us.





In [3]:
import tensorflow as tf
from tensorflow import keras
from collections import Counter
import tensorflow_datasets as tfds



In [4]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [5]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [6]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [7]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [8]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [9]:
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [10]:
len(vocabulary)

53893

In [11]:
vocab_size = 10000
truncated_vocabulary = [
word for word, count in vocabulary.most_common()[:vocab_size]]

In [12]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [13]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

In [14]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch


train_set = datasets["train"].batch(32).map(preprocess)
test_set = datasets["test"].batch(32).map(preprocess)

train_set = train_set.map(encode_words).cache().prefetch(1)
test_set = test_set.map(encode_words).cache().prefetch(1)

In [19]:
embed_dim = 16
hidden_dim = 64
dropout_rate = 0.5
nonlinearity = 'tanh'
learning_rate = 1e-3
epochs = 10


model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_dim,
                           mask_zero=True, input_shape=[None]),
    keras.layers.SimpleRNN(hidden_dim, activation=nonlinearity),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
metrics=["accuracy"])
history = model.fit(train_set, epochs=epochs)

Epoch 1/10
782/782 [==============================] - 29s 34ms/step - loss: 0.6864 - accuracy: 0.5371
Epoch 2/10
782/782 [==============================] - 31s 40ms/step - loss: 0.5187 - accuracy: 0.7552
Epoch 3/10
782/782 [==============================] - 25s 32ms/step - loss: 0.3759 - accuracy: 0.8434
Epoch 4/10
782/782 [==============================] - 25s 32ms/step - loss: 0.2422 - accuracy: 0.9060
Epoch 5/10
782/782 [==============================] - 25s 32ms/step - loss: 0.1664 - accuracy: 0.9385
Epoch 6/10
782/782 [==============================] - 26s 33ms/step - loss: 0.1184 - accuracy: 0.9574
Epoch 7/10
782/782 [==============================] - 25s 32ms/step - loss: 0.0931 - accuracy: 0.9681
Epoch 8/10
782/782 [==============================] - 25s 32ms/step - loss: 0.0547 - accuracy: 0.9817
Epoch 9/10
782/782 [==============================] - 26s 34ms/step - loss: 0.0586 - accuracy: 0.9802
Epoch 10/10
782/782 [==============================] - 25s 32ms/step - loss: 0.036

In [20]:
print("Train set\n",model.evaluate(train_set, steps=train_size // 32, batch_size=32))

print("Test set\n",model.evaluate(test_set, steps=test_size // 32, batch_size=32))

781/781 [==============================] - 9s 10ms/step - loss: 0.0385 - accuracy: 0.9860
Train set
 [0.038450371474027634, 0.9859955310821533]
781/781 [==============================] - 10s 12ms/step - loss: 1.7259 - accuracy: 0.6605
Test set
 [1.72585928440094, 0.6604913473129272]


# LSTM Implementation

Starter code taken from readings provided to us

Since we preprocess the data and create the final training set above, all that is left to do is to simply create the LSTM model. For this, a simple substitution in our earlier code is enough.

In [21]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_dim, mask_zero=True, input_shape=[None]),
    keras.layers.LSTM(hidden_dim, activation=nonlinearity),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=epochs)

Epoch 1/10
782/782 [==============================] - 53s 58ms/step - loss: 0.5408 - accuracy: 0.7157
Epoch 2/10
782/782 [==============================] - 47s 60ms/step - loss: 0.3827 - accuracy: 0.8358
Epoch 3/10
782/782 [==============================] - 47s 60ms/step - loss: 0.3224 - accuracy: 0.8714
Epoch 4/10
782/782 [==============================] - 47s 60ms/step - loss: 0.2732 - accuracy: 0.8969
Epoch 5/10
782/782 [==============================] - 45s 58ms/step - loss: 0.2268 - accuracy: 0.9164
Epoch 6/10
782/782 [==============================] - 46s 59ms/step - loss: 0.1914 - accuracy: 0.9329
Epoch 7/10
782/782 [==============================] - 47s 60ms/step - loss: 0.1658 - accuracy: 0.9428
Epoch 8/10
782/782 [==============================] - 47s 60ms/step - loss: 0.1405 - accuracy: 0.9526
Epoch 9/10
782/782 [==============================] - 46s 58ms/step - loss: 0.1267 - accuracy: 0.9556
Epoch 10/10
782/782 [==============================] - 48s 61ms/step - loss: 0.103

In [22]:
print("Train set\n",model.evaluate(train_set, steps=train_size // 32, batch_size=32))

print("Test set\n",model.evaluate(test_set, steps=test_size // 32, batch_size=32))

781/781 [==============================] - 14s 15ms/step - loss: 0.1675 - accuracy: 0.9417
Train set
 [0.16747130453586578, 0.9416613578796387]
781/781 [==============================] - 15s 19ms/step - loss: 1.1215 - accuracy: 0.7221
Test set
 [1.1215111017227173, 0.7220710515975952]
